# Setup

In [3]:
import os, sys
chapter = "chapter1_transformer_interp"
repo = "ARENA_3.0"
chapter_dir = r"./" if chapter in os.listdir() else os.getcwd().split(chapter)[0]
sys.path.append(chapter_dir + f"{chapter}/exercises")

import os
os.environ["ACCELERATE_DISABLE_RICH"] = "1"
import sys
import torch as t
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch import Tensor
from torch.utils.data import DataLoader
import numpy as np
import einops
from ipywidgets import interact
import plotly.express as px
from ipywidgets import interact
from pathlib import Path
import itertools
import random
from IPython.display import display
from jaxtyping import Float, Int, Bool, Shaped, jaxtyped
from typing import List, Union, Optional, Tuple, Callable, Dict
import typeguard
from functools import partial
# from torcheval.metrics.functional import multiclass_f1_score
from sklearn.metrics import f1_score as multiclass_f1_score
import copy
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
import datasets
from IPython.display import HTML
import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import HookedRootModule, HookPoint
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache
from tqdm.notebook import tqdm
from dataclasses import dataclass
from rich import print as rprint
import pandas as pd

# Make sure exercises are in the path
# exercises_dir = Path(f"{os.getcwd().split(chapter)[0]}/{chapter}/exercises").resolve()
# section_dir = exercises_dir / "part6_othellogpt"
# section_dir = "interpretability"
# if str(exercises_dir) not in sys.path: sys.path.append(str(exercises_dir))

from plotly_utils import imshow
from neel_plotly import scatter, line
from generate_patches import generate_patch
from pprint import pprint
from utils import plot_game
from training_utils import get_state_stack_num_flipped
from utils import plot_probe_outputs
from utils import seq_to_state_stack
from utils import VisualzeBoardArguments
from utils import visualize_game

# import part6_othellogpt.tests as tests

t.manual_seed(42)

device = t.device("cuda" if t.cuda.is_available() else "cpu")

MAIN = __name__ == "__main__"

cfg = HookedTransformerConfig(
    n_layers = 8,
    d_model = 512,
    d_head = 64,
    n_heads = 8,
    d_mlp = 2048,
    d_vocab = 61,
    n_ctx = 59,
    act_fn="gelu",
    normalization_type="LNPre",
    device=device,
)
model = HookedTransformer(cfg)

sd = utils.download_file_from_hf("NeelNanda/Othello-GPT-Transformer-Lens", "synthetic_model.pth")
# champion_ship_sd = utils.download_file_from_hf("NeelNanda/Othello-GPT-Transformer-Lens", "championship_model.pth")
model.load_state_dict(sd)

# An example input
sample_input = t.tensor([[
    20, 19, 18, 10,  2,  1, 27,  3, 41, 42, 34, 12,  4, 40, 11, 29, 43, 13, 48, 56,
    33, 39, 22, 44, 24,  5, 46,  6, 32, 36, 51, 58, 52, 60, 21, 53, 26, 31, 37,  9,
    25, 38, 23, 50, 45, 17, 47, 28, 35, 30, 54, 16, 59, 49, 57, 14, 15, 55, 7
]]).to(device)

# The argmax of the output (ie the most likely next move from each position)
sample_output = t.tensor([[
    21, 41, 40, 34, 40, 41,  3, 11, 21, 43, 40, 21, 28, 50, 33, 50, 33,  5, 33,  5,
    52, 46, 14, 46, 14, 47, 38, 57, 36, 50, 38, 15, 28, 26, 28, 59, 50, 28, 14, 28,
    28, 28, 28, 45, 28, 35, 15, 14, 30, 59, 49, 59, 15, 15, 14, 15,  8,  7,  8
]]).to(device)

assert (model(sample_input).argmax(dim=-1) == sample_output.to(device)).all()

# os.chdir(section_dir)
section_dir = Path.cwd()
sys.path.append(str(section_dir))
print(section_dir.name)

OTHELLO_ROOT = (section_dir / "othello_world").resolve()
OTHELLO_MECHINT_ROOT = (OTHELLO_ROOT / "mechanistic_interpretability").resolve()

# if not OTHELLO_ROOT.exists():
#     !git clone https://github.com/likenneth/othello_world

sys.path.append(str(OTHELLO_MECHINT_ROOT))

from mech_interp_othello_utils import (
    plot_board,
    plot_single_board,
    plot_board_log_probs,
    to_string,
    to_int,
    int_to_label,
    string_to_label,
    OthelloBoardState
)

# Load board data as ints (i.e. 0 to 60)
board_seqs_int = t.tensor(np.load(OTHELLO_MECHINT_ROOT / "board_seqs_int_small.npy"), dtype=t.long)
# Load board data as "strings" (i.e. 0 to 63 with middle squares skipped out)
board_seqs_string = t.tensor(np.load(OTHELLO_MECHINT_ROOT / "board_seqs_string_small.npy"), dtype=t.long)

assert all([middle_sq not in board_seqs_string for middle_sq in [27, 28, 35, 36]])
assert board_seqs_int.max() == 60

num_games, length_of_game = board_seqs_int.shape

# Define possible indices (excluding the four center squares)
stoi_indices = [i for i in range(64) if i not in [27, 28, 35, 36]]

# Define our rows, and the function that converts an index into a (row, column) label, e.g. `E2`
alpha = "ABCDEFGH"

def to_board_label(i):
    return f"{alpha[i//8]}{i%8}"

# Get our list of board labels
board_labels = list(map(to_board_label, stoi_indices))
full_board_labels = list(map(to_board_label, range(64)))

def plot_square_as_board(state, diverging_scale=True, **kwargs):
    """Takes a square input (8 by 8) and plot it as a board. Can do a stack of boards via facet_col=0"""
    kwargs = {
        "y": [i for i in alpha],
        "x": [str(i) for i in range(8)],
        "color_continuous_scale": "RdBu" if diverging_scale else "Blues",
        "color_continuous_midpoint": 0. if diverging_scale else None,
        "aspect": "equal",
        **kwargs
    }
    imshow(state, **kwargs)

start = 30000
num_games = 50
focus_games_int = board_seqs_int[start : start + num_games]
focus_games_string = board_seqs_string[start: start + num_games]

focus_logits, focus_cache = model.run_with_cache(focus_games_int[:, :-1].to(device))
focus_logits.shape

def one_hot(list_of_ints, num_classes=64):
    out = t.zeros((num_classes,), dtype=t.float32)
    out[list_of_ints] = 1.
    return out

focus_states = np.zeros((num_games, 60, 8, 8), dtype=np.float32)
focus_valid_moves = t.zeros((num_games, 60, 64), dtype=t.float32)

for i in (range(num_games)):
    board = OthelloBoardState()
    for j in range(60):
        board.umpire(focus_games_string[i, j].item())
        focus_states[i, j] = board.state
        focus_valid_moves[i, j] = one_hot(board.get_valid_moves())

print("focus states:", focus_states.shape)
print("focus_valid_moves", tuple(focus_valid_moves.shape))

# full_linear_probe = t.load(OTHELLO_MECHINT_ROOT / "main_linear_probe.pth", map_location=device)

linear_probe2 = t.load("probes/linear/resid_6_linear.pth")

rows = 8
cols = 8
options = 3
assert linear_probe2.shape == (1, cfg.d_model, rows, cols, options)

black_to_play_index = 0
white_to_play_index = 1
blank_index = 0
their_index = 1
my_index = 2

# Creating values for linear probe (converting the "black/white to play" notation into "me/them to play")

from utils import *

/hpi/fs00/home/jim.maar/miniconda3/envs/othello-env/lib/python3.11/site-packages/accelerate/utils/imports.py:274: UserWarning: `ACCELERATE_DISABLE_RICH` is deprecated and will be removed in v0.22.0 and deactivated by default. Please use `ACCELERATE_ENABLE_RICH` if you wish to use `rich`.
  warnings.warn(


interpretability
interpretability
focus states: (50, 60, 8, 8)
focus_valid_moves (50, 60, 64)


# Code

In [4]:
W_E_pos = model.W_E_pos.clone()[0:59]

In [5]:
def pos_emb_hook(activation : Float[Tensor, "batch pos d_model"], hook : HookPoint):
    print(activation.shape)
    print(W_E_pos.shape)
    activation = einops.repeat(W_E_pos, 'pos d_model -> batch pos d_model', batch=activation.shape[0])
    print(activation.shape)
    return activation

In [6]:
from circuitsvis.attention import attention_patterns

In [16]:
def plot_attn_pattern(
        cache : ActivationCache = focus_cache,
        game_idx : int = 0,
        games_string : Int[Tensor, "batch seq"] = focus_games_string,
        layer : Int = 0,
        heads : List[Int] = list(range(8)),
        how_many : Int = 59,
    ):
    attention_pattern = cache["pattern", layer][0]
    attention_pattern = attention_pattern[heads, :how_many, :how_many]
    str_tokens = games_string[game_idx, :how_many].tolist()	
    str_tokens = [f" {i}: {string_to_label(str_tokens[i])}" for i in range(len(str_tokens))]
    display(attention_patterns(
        tokens=str_tokens,
        attention=attention_pattern,
        attention_head_names=[f"L{layer}H{i}" for i in heads],
    ))

for layer in range(8):
    plot_attn_pattern(how_many = 50, layer = layer)
# Fix Bug AAAAAG

In [ ]:
(0, 0)
heads_dict = {
    "mine_far" : [(0, 3), (0, 6)],
    "yours_far" : [(0, 0), (0, 7)],
    "mine_close" : [],
    "yours_close" : [(0, 2)],
    "mine_last" : [],
    "yours_last" : [],
    "first" : [],
    "stripe" : [(0, 1), (0, 4), (0, 5)],
}

In [8]:
model.reset_hooks()
patched_cache = model.add_caching_hooks()
_ = model.run_with_hooks(
    focus_games_int[:, :-1].to(device),
    fwd_hooks = [(utils.get_act_name("resid_pre", 0), pos_emb_hook)],
)
patched_cache = ActivationCache(patched_cache, model)

torch.Size([50, 59, 512])
torch.Size([59, 512])
torch.Size([50, 59, 512])


In [9]:
plot_attn_pattern(cache=patched_cache, how_many = 20)

In [17]:
# Ich will Heads aufteilen in MINE und YOURS attending und vielleicht wie far Back. Dann gabs auch noch First Token und Last token
# attention to itself, last, mine, yours, first, how far back (how do I measure this what is the function with which this decreases?)
# First do the mine / yours distinction then look at the function
THRESHOLDS = {
    "mine" : 40,
    "yours" : 40,
    "last_mine" : 13,
    "last_yours" : 18,
    "first" : 24,
    "close" : 18,
    "far" : 15,
}
CLOSE_THRESH = 10
def get_mine_their_score(cache : ActivationCache, layer : Int, head : Int):
    attention_pattern : Float[Tensor, "batch head pos_q pos_k"]= cache["pattern", layer]
    attention_pattern : Float[Tensor, "pos_q pos_k"]= attention_pattern.mean(dim=0)[head]
    scores = {
        "mine": 0,
        "yours": 0,
        "last_mine": 0,
        "last_yours": 0,
        "first": 0,
        "close": 0,
        "far" : 0,
        "var" : 0,
    }
    k_scores = [0 for k in range(attention_pattern.shape[1])]

    for q in range(attention_pattern.shape[0]):
        for k in range(q):
            attention_score = attention_pattern[q, k].item()
            if k == 0:
                scores["first"] += attention_score
            if k == q - 1:
                scores["last_yours"] += attention_score
            if k == q - 2:
                scores["last_mine"] += attention_score
            if k % 2 == q % 2:
                scores["mine"] += attention_score
            if k % 2 != q % 2:
                scores["yours"] += attention_score
            if abs(q - k) < CLOSE_THRESH and k != q - 1 and k != q - 2:
                scores["close"] += attention_score
            if abs(q - k) > CLOSE_THRESH and k != 0:
                scores["far"] += attention_score
            k_scores[k] += attention_score
    scores["var"] = np.var(k_scores)
    for key in scores:
        scores[key] = round(scores[key])
    return scores

def classify_heads(cache : ActivationCache):
    heads_dict = {
        "mine_far" : [],
        "yours_far" : [],
        "mine_close" : [],
        "yours_close" : [],
        "mine_last" : [],
        "yours_last" : [],
        "first" : [],
        "stripe" : [],
    }
    for layer in range(8):
        for head in range(8):
            scores = get_mine_their_score(cache, layer, head)
            if scores["mine"] > THRESHOLDS["mine"]:
                is_mine = True
            else:
                is_mine = False
            if scores["yours"] > THRESHOLDS["yours"]:
                is_yours = True
            else:
                is_yours = False
            if scores["far"] > THRESHOLDS["far"]:
                is_far = True
            else:
                is_far = False
            if scores["close"] > THRESHOLDS["close"]:
                is_close = True
            else:
                is_close = False
            if scores["last_mine"] > THRESHOLDS["last_mine"]:
                is_last_mine = True
            else:
                is_last_mine = False
            if scores["last_yours"] > THRESHOLDS["last_yours"]:
                is_last_yours = True
            else:
                is_last_yours = False
            if scores["first"] > THRESHOLDS["first"]:
                is_first = True
            else:
                is_first = False
            if (layer, head) in [(0, 1), (0, 4), (0, 5)]:
                heads_dict["stripe"] += [(layer, head)]
                continue
            if is_far and is_close:
                print(f"far and close: L{layer}H{head}")
            if is_mine and is_yours:
                print(f"mine and yours: L{layer}H{head}")
            if is_mine and is_far:
                heads_dict["mine_far"] += [(layer, head)]
            if is_mine and is_close:
                heads_dict["mine_close"] += [(layer, head)]
            if is_yours and is_far:
                heads_dict["yours_far"] += [(layer, head)]
            if is_yours and is_close:
                heads_dict["yours_close"] += [(layer, head)]
            if is_last_mine:
                heads_dict["mine_last"] += [(layer, head)]
            if is_last_yours:
                heads_dict["yours_last"] += [(layer, head)]
            if is_first:
                heads_dict["first"] += [(layer, head)]            
    return heads_dict

'''for layer in range(8):
    for head in range(8):
        scores = get_mine_their_score(focus_cache, layer, head)
        print(f"Layer {layer} Head {head}, Score {scores}")'''


heads_dict = classify_heads(focus_cache)

# pickle heads_dict
import pickle
with open("heads_dict.pkl", "wb") as f:
    pickle.dump(heads_dict, f)

print(heads_dict)

    

far and close: L1H0
{'mine_far': [(0, 3), (0, 6), (1, 0)], 'yours_far': [(0, 0), (0, 7)], 'mine_close': [(1, 0), (1, 4), (1, 5), (1, 6), (2, 0), (2, 6), (2, 7), (3, 0), (4, 4), (4, 6)], 'yours_close': [(0, 2), (1, 2), (1, 3), (2, 1), (2, 3), (2, 4), (2, 5), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (4, 0), (4, 1), (4, 2), (4, 3), (4, 7), (5, 0), (5, 3), (5, 4), (5, 5), (5, 7), (6, 2), (6, 5)], 'mine_last': [(1, 5), (3, 0)], 'yours_last': [(0, 2), (2, 3), (4, 5), (5, 7)], 'first': [(1, 1), (1, 7), (3, 2), (4, 0), (4, 3), (5, 0), (5, 3), (5, 4), (6, 0), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5), (6, 6), (6, 7), (7, 1), (7, 4), (7, 5), (7, 6)], 'stripe': [(0, 1), (0, 4), (0, 5)]}
